In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.metrics import balanced_accuracy_score,precision_score,recall_score,f1_score, roc_auc_score

In [2]:
df = pd.read_csv('../../data/crawler/unified-events-statistics.csv')
def qtd_eventos(integer):
    return 45 - integer

qtd_eventos = list(map(qtd_eventos, df.isnull().sum(axis=1).tolist()))
df['qtd_eventos'] = qtd_eventos
df = df[df.qtd_eventos > 25]
df = df.replace(['BLUE: first_blood',
    'BLUE: dragon',
    'BLUE: herald',
    'BLUE: first_tower_top',
    'BLUE: first_tower_mid',
    'BLUE: first_tower_bot',
    'BLUE: second_tower_top',
    'BLUE: second_tower_mid',
    'BLUE: second_tower_bot',
    'BLUE: third_tower_top',
    'BLUE: third_tower_mid',
    'BLUE: third_tower_bot',
    'BLUE: inhibitor_top',
    'BLUE: inhibitor_mid',
    'BLUE: inhibitor_bot',
    'BLUE: baron',
    'BLUE: elder_dragon',
    'BLUE: nexus_tower',
    'BLUE: nexus',
    'RED: first_blood',
    'RED: dragon',
    'RED: herald',
    'RED: first_tower_top',
    'RED: first_tower_mid',
    'RED: first_tower_bot',
    'RED: second_tower_top',
    'RED: second_tower_mid',
    'RED: second_tower_bot',
    'RED: third_tower_top',
    'RED: third_tower_mid',
    'RED: third_tower_bot',
    'RED: inhibitor_top',
    'RED: inhibitor_mid',
    'RED: inhibitor_bot',
    'RED: baron',
    'RED: elder_dragon',
    'RED: nexus_tower',
    'RED: nexus'], range(38))
df

,golId,game,event1,event2,event3,event4,event5,event6,event7,event8,...,redMidWR,redMidKDA,redAdcGP,redAdcWR,redAdcKDA,redSupportGP,redSupportWR,redSupportKDA,result,qtd_eventos
0,36864,ESPORTSTMNT02_2556988,19,1,2,1,3,24,2,23,...,0.71,4.2,1,0.00,2.5,1,1.00,7.0,1,28
2,36866,ESPORTSTMNT02_2557390,19,2,20,24,2,4,20,3,...,0.80,4.8,1,0.00,0.8,5,0.60,3.2,0,31
7,36889,ESPORTSTMNT02_2557053,19,1,2,5,1,20,21,4,...,0.00,0.0,1,0.00,0.4,0,0.00,0.0,1,32
11,36894,ESPORTSTMNT02_2577994,0,20,2,1,5,3,6,20,...,0.00,0.0,1,1.00,12.0,13,0.46,1.9,1,30
13,36896,ESPORTSTMNT02_2576974,0,20,2,1,4,21,20,3,...,0.50,3.3,9,0.44,4.3,8,0.75,2.1,0,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4075,44351,ESPORTSTMNT02_3045864,0,21,20,21,5,4,22,23,...,0.52,2.8,12,0.50,5.4,10,0.90,3.9,1,27
4080,44356,ESPORTSTMNT02_3056652,19,2,3,20,24,23,21,20,...,0.46,2.9,58,0.74,4.0,65,0.62,3.5,1,26
4081,44356,ESPORTSTMNT02_3056652,19,2,3,20,24,23,21,20,...,0.46,2.9,58,0.74,4.0,65,0.62,3.5,1,26
4100,44374,ESPORTSTMNT02_3057882,19,2,1,23,2,4,7,20,...,0.81,10.1,29,0.62,3.7,7,0.43,5.3,1,28


In [3]:
y = df['result'].copy()
X = df.drop(['golId','result','game','event26','event27','event28','event29','event30','event31','event32','event33','event34','event35','event36','event37','event38','event39','event40','event41','event42','event43','event44','event45'],axis=1)
X

,event1,event2,event3,event4,event5,event6,event7,event8,event9,event10,...,redMidGP,redMidWR,redMidKDA,redAdcGP,redAdcWR,redAdcKDA,redSupportGP,redSupportWR,redSupportKDA,qtd_eventos
0,19,1,2,1,3,24,2,23,6,1,...,14,0.71,4.2,1,0.00,2.5,1,1.00,7.0,28
2,19,2,20,24,2,4,20,3,6,1,...,5,0.80,4.8,1,0.00,0.8,5,0.60,3.2,31
7,19,1,2,5,1,20,21,4,1,3,...,0,0.00,0.0,1,0.00,0.4,0,0.00,0.0,32
11,0,20,2,1,5,3,6,20,2,4,...,0,0.00,0.0,1,1.00,12.0,13,0.46,1.9,30
13,0,20,2,1,4,21,20,3,5,22,...,2,0.50,3.3,9,0.44,4.3,8,0.75,2.1,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4075,0,21,20,21,5,4,22,23,1,3,...,31,0.52,2.8,12,0.50,5.4,10,0.90,3.9,27
4080,19,2,3,20,24,23,21,20,15,5,...,13,0.46,2.9,58,0.74,4.0,65,0.62,3.5,26
4081,19,2,3,20,24,23,21,20,15,5,...,13,0.46,2.9,58,0.74,4.0,65,0.62,3.5,26
4100,19,2,1,23,2,4,7,20,3,24,...,26,0.81,10.1,29,0.62,3.7,7,0.43,5.3,28


In [4]:
def preprocess_input(X,y):
    X = X.copy()
    X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=42, stratify=y)
    scaler = StandardScaler()   
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train,X_test,y_train,y_test
X_train,X_test,y_train,y_test = preprocess_input(X,y)

In [5]:
from sklearn.ensemble import RandomForestClassifier

n_estimators = [50]
max_depth = [10]
min_samples_split = [8]
min_samples_leaf = [3] 

hyperF = dict(n_estimators = n_estimators, max_depth = max_depth,  
              min_samples_split = min_samples_split, 
             min_samples_leaf = min_samples_leaf)
forest = RandomForestClassifier(random_state=42)
gridF = GridSearchCV(forest,hyperF,cv=3,verbose=1,n_jobs=-1)
bestF = gridF.fit(X_train, y_train)

Fitting 3 folds for each of 225 candidates, totalling 675 fits


In [6]:
print("Best: %f using %s" % (bestF.best_score_, bestF.best_params_))

Best: 0.893252 using {'max_depth': 10, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 50}


In [9]:
forestOpt = RandomForestClassifier(random_state=42,max_depth=10,n_estimators=50,min_samples_split=8,min_samples_leaf=3)
                                   
modelOpt = forestOpt.fit(X_train, y_train)

scores_list = []
scores_list.append({
    'Balanced Accuracy': balanced_accuracy_score(y_test,modelOpt.predict(X_test)),
    'Precision':  precision_score(y_test,modelOpt.predict(X_test)),
    'Recall': recall_score(y_test,modelOpt.predict(X_test)),
    'F1-Score': f1_score(y_test,modelOpt.predict(X_test)),
    'ROC/AUC Score': roc_auc_score(y_test,modelOpt.predict(X_test))
})
scores = pd.DataFrame(scores_list)

In [10]:
scores

,Balanced Accuracy,Precision,Recall,F1-Score,ROC/AUC Score
0,0.913215,0.905172,0.913043,0.909091,0.913215
